In [5]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
from openai import OpenAI
import os
load_dotenv()
index_name = "bezzie-index"
pinecone = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [6]:
index_name = "bezzie-test-index"
def create_index():
    if not pinecone.has_index(index_name):
        pinecone.create_index(
            name=index_name,
            metric="cosine",
            dimension=1536,
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
            tags={
                "environment": "development"
            }
        )
    else:
        return pinecone.describe_index(name=index_name)
indexRes = create_index()

In [7]:
def createEmbedding():
    res = client.embeddings.create(
        model="text-embedding-3-small",
        input="The Person is little bit depressed and going through tough time in work and his social life including his family and friends that are avoiding him due to his constant failures, he is also going through a hard time due to no job interview calls",
        dimensions=1536,
        encoding_format='float'
    )
    
    return res
embd = createEmbedding()

In [8]:
for i in embd.data:
    print(i.embedding)

[-0.012493487, -0.0052371384, 0.048949093, 0.041872703, -0.0139575675, -0.04704579, 0.019411268, 0.032185372, 0.016312297, -0.0236327, -0.030623686, -0.04233633, -0.051877253, 0.010089955, 0.015726665, 0.033869065, 0.041701894, 0.01564126, -0.026426654, 0.03469871, 0.024303738, 0.059002448, 0.017361555, 0.0040780744, 0.027768727, 0.0074241087, 0.003980469, 0.018093595, 0.0745217, 0.015494852, 0.036772825, -0.042555943, 0.0055970578, 0.03238058, -0.028281156, 0.0016028632, 0.014140578, -0.0033551846, 0.009138303, 0.010529179, 0.027158694, -0.011901755, 0.011578437, 0.01726395, -0.007234998, -0.009412818, -0.016263494, -0.0132255275, 0.039700985, -0.020289717, -0.030038053, 0.0140795745, 0.03286861, 0.06978784, -0.019862693, -0.025328593, -0.006777473, -0.05138923, -0.022473637, 0.006923881, 0.05758717, -0.032209773, 0.019874893, -0.0199847, -0.02325448, -0.028695978, 0.024864968, 0.016214693, -0.021302372, -0.04140908, 0.043019567, 0.016958933, 0.0001332542, 0.039212957, 0.011895655, 0.

In [9]:
index_host = indexRes.index.host

In [10]:
def upsert_embedding(values:list[list[float]]):
    index = pinecone.Index(host=index_host)
    data = []
    for i in values:
        data.append({
            "id": "uid",
            "values": i,
            "metadata": {"type": "mildly depressed"}
        })
    
    res = index.upsert(
        vectors=data,
        namespace="bezzie-testing"
    )
    return res
values = [embd.data[0].embedding]
res = upsert_embedding(values)
print(res)

{'upserted_count': 1}
